In [1]:
%load_ext autoreload
%autoreload 2

import pygame
import time
import numpy as np

from game import agent_by_mark, ConnectFourEnv, next_mark # possibly need to set export DISPLAY=:0 in terminal?
from play_human import HumanAgent
# does this need to run?
# pygame.display.list_modes()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## TO RUN VIDEOSTUFF ON WINDOWS / WSL:
- 1: start xming
- 2: in __terminal__ (not VSCode!), prior to launching notebook, run export DISPLAY=:0

In [2]:
# Define parameters

NUM_ROWS = 6
NUM_COLS = 7

CELL_RADIUS = 20
CELL_PADDING = 15

BOARD_HEIGHT = NUM_ROWS * (2*CELL_RADIUS + CELL_PADDING) + CELL_PADDING
BOARD_WIDTH = NUM_COLS * (2*CELL_RADIUS + CELL_PADDING) + CELL_PADDING

#VELOCITY = 1
MARKER_RADIUS = CELL_RADIUS

BOARD_COLOR = pygame.Color('blue')
CELL_COLOR = pygame.Color('black')
MARKER_1_COLOR = pygame.Color('yellow')
MARKER_2_COLOR = pygame.Color('red')

DROP_PANEL_WIDTH = BOARD_WIDTH
DROP_PANEL_HEIGHT = 80
DROP_PANEL_COLOR = pygame.Color('black')

SCREEN_WIDTH = BOARD_WIDTH
SCREEN_HEIGHT = BOARD_HEIGHT + DROP_PANEL_HEIGHT

In [3]:
pygame.init()

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) # returns the surface where you can draw stuff
pygame.draw.rect(screen, DROP_PANEL_COLOR, pygame.Rect((0,0), (DROP_PANEL_WIDTH, DROP_PANEL_HEIGHT)))
pygame.draw.rect(screen, BOARD_COLOR, pygame.Rect((0,DROP_PANEL_HEIGHT), (SCREEN_WIDTH, SCREEN_HEIGHT)))
#pygame.display.flip()


# draw empty cells

def draw_empty_board():
    global screen
    global CELL_RADIUS, CELL_PADDING, CELL_COLOR
    global NUM_ROWS, NUM_COLS
    
    IDXS_TO_PIXEL_CORDS_MAP = dict()

    y_pos = DROP_PANEL_HEIGHT + CELL_RADIUS + CELL_PADDING
    for row_idx in range(NUM_ROWS):
        x_pos = CELL_RADIUS + CELL_PADDING
        for col_idx in range(NUM_COLS):
            IDXS_TO_PIXEL_CORDS_MAP[(row_idx, col_idx)] = (x_pos, y_pos)
            pygame.draw.circle(screen, CELL_COLOR, (x_pos, y_pos), CELL_RADIUS)
            x_pos += 2 * CELL_RADIUS + CELL_PADDING
        y_pos += 2 * CELL_RADIUS + CELL_PADDING
    
    return IDXS_TO_PIXEL_CORDS_MAP

IDXS_TO_PIXEL_CORDS_MAP = draw_empty_board()


def draw_board(board):
    global IDXS_TO_PIXEL_CORDS_MAP
    
    if isinstance(board, tuple): # undo board flattening
        board = np.array(board).reshape(6, 7)
    else:
        board = board.reshape(6,7)
        
    color_map = {0: CELL_COLOR, -1: MARKER_1_COLOR, 1: MARKER_2_COLOR}
    
    for row_index in range(board.shape[0]):
        for col_index in range(board.shape[1]):            
            marker_value = board[row_index, col_index]
            marker_color = color_map[int(marker_value)]
            pygame.draw.circle(screen, marker_color, IDXS_TO_PIXEL_CORDS_MAP[(row_index, col_index)], CELL_RADIUS)




In [4]:
time

<module 'time' (built-in)>

In [5]:
# run HUM vs AI game
from train_sb_agent import ConnectFourA2C

env = ConnectFourEnv(show_number=False,
                     interactive_mode = True,
                     opponent = 'random')
#agents = [
#    HumanAgent('O'), #HumanAgent('X')]
#    ConnectFourA2C.load('a2c_agent_50k_vs100kA2C')
#]

human_agent = HumanAgent('O')
ai_agent = ConnectFourA2C.load('a2c_agent_50k_vs100kA2C')
    
state = env.reset()
board, mark = state
done = False
draw_board(board)


try:
    pygame.display.flip()
    while True:
        e = pygame.event.poll()
        if e.type == pygame.QUIT:
            break

        pygame.display.flip()

        #checks if a mouse is clicked
        if e.type == pygame.MOUSEBUTTONDOWN: 
            # Human plays        
            CLICK_ACTION = int(mouse_x // (SCREEN_WIDTH/NUM_COLS))
            env.show_turn(True, mark)
            ava_actions = env.available_actions()
            action, action_is_valid = human_agent.pygame_act(ava_actions,CLICK_ACTION)

            if not action_is_valid:
                print('Invalid action not supported, exiting!')
                pygame.quit()
                break

            state, reward, done, info = env._step(action)
            observation, mark = state
            if done:
                env.show_result(True, mark, reward)
                break

            # AI plays
            action = ai_agent.predict_with_invalid_mask(observation, env = env)
            state, rewards, done, info = env._step(action)
            observation, mark = state
            draw_board(observation)
            if done:
                env.show_result(True, mark, reward)
                break

        mouse_x, mouse_y = pygame.mouse.get_pos()
    draw_board(observation)
    time.sleep(5)
    pygame.quit()

except Exception as e:
    
    pygame.quit()
    raise e

/home/olof/anaconda3/envs/FourInARow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


O's turn.
O's turn.
O's turn.
O's turn.
O's turn.
The winner is player: -1
The winner is player: -1
==== Finished: Winner is 'O'! ====

